# UDP Chat
#### [V. Hunter Adams](https://vanhunteradams.com)

***

UDP (User Datagram) lacks some of the nice features of TCP (no gaurantees with regard to packet delivery, and no guarantee that packets will be delivered in the correct order), but it also lacks much of the overhead associated with TCP. This makes it fast, and it also makes it a very nice starting point for working with LWIP! This webpage shows how to setup a simple chat capability between a PC and and Pi Pico W.

***

## Some background

Some understanding of the data structures that LWIP uses to manage network connections and for communicating network data through the stack is helpful for assembling and debugging these programs. There are two data structures in particular that are useful to understand: **protocol control blocks (PCB's)** and **packet buffers (pbuf's)**. Let's consider each in turn.

#### Protocol control blocks

Your application code may be managing multiple network connections at the same time, perhaps receiving information over one port from one remote IP address and sending information out to some other destination address over a different port. LWIP uses a **protocol control block (PCB)** to manage each of these network connections. Each PCB is a struct that contains all of the information of relevance for a particular connection. In the case of a UDP protocol control block, this information includes things like the local IP, remote IP, local and remote ports, and a pointer to a receive callback function (plus some flags for settings).

Perhaps it stands to reason that the protocol control block for [a TCP connection](https://github.com/lwip-tcpip/lwip/blob/ca0395c5ae54273b4fd542ef25b57d4b759678f8/src/include/lwip/tcp.h#L242) maintains a lot more state than [that for UDP](https://github.com/lwip-tcpip/lwip/blob/ca0395c5ae54273b4fd542ef25b57d4b759678f8/src/include/lwip/udp.h#L81). UDP is connectionless and therefore receives no receipt of packet acknowledgements. It has no error correction (beyond a checksum in each packet) and therefore maintains no state associated with retransmission attempts.

#### Packet buffers

[Packet buffers (PBUF's)](https://github.com/lwip-tcpip/lwip/blob/ca0395c5ae54273b4fd542ef25b57d4b759678f8/src/include/lwip/pbuf.h#L186) are used to shuffle data through the LWIP stack. These are the containers that hold the packet data, and some metainformation about that packet. Each contains a pointer to the payload (the actual data), a field that indicates the total length of the buffer, a field that contains the length of this *particular* buffer, and a pointer to another PBUF called `next`. In the event that the packet data exceeds that which can be held in a single buffer, this pointer allows for us to chain PBUF's in a linked list. Other fields include the PBUF type (does this PBUF live in RAM, ROM, etc.?), some flags, and a reference count. The reference count tracks the number of users of this PBUF so that it's not freed to soon (more on that later). The final fields include one to hold the netif index for an incoming packet, and a place where a user could store custom data on a PBUF.

This is how LWIP organizes packet information! When we want to send a packet, we will allocate a PBUF, populate its payload field, and send it out by way of a protocol control block. When we receive a packet, LWIP will emit that packet to our callback function *as a PBUF*. As we'll see, it's our responsibility to *free* each PBUF once we've finished using it. We interact with these PBUFF's largely by way of the [PBUF API](https://www.nongnu.org/lwip/2_1_x/group__pbuf.html).


***

## Receiving UDP on the Pico W

Setting up our device to *receive* UDP packets involves the following steps:

1. Creating a protocol control block
2. *Binding* that protocol control block to a specific local IP and port. This way, any message sent to our IP address at that specific port will be routed to this particular PCB.
3. Setup a receive callback function for this PCB that will process incoming messages sent to the IP/port that we've bound.
4. Within that receive callback function, which has a standard set of arguments that includes a PBUF in which the incoming packet information is stored, process the packet payload and then free the PBUF.

Let us examine each of these steps.

#### Creating a protocol control block

We can declare a UDP protocol control block globally, and then we'll assign its value in an initialization function. Declaring the PCB (or, more specifically, a *pointer* to the PCB) looks like the following:

```c
// Protocol control block for UDP receive connection
static struct udp_pcb *udp_rx_pcb;
```

And the initializing this pcb looks like this:

```c
// Initialize the RX protocol control block
udp_rx_pcb  = udp_new_ip_type(IPADDR_TYPE_ANY);
```

Under the hood, `udp_new_ip_type` makes a call to the function `udp_new()`, which allocates a new udp protocol control block by way of `(struct udp_pcb *)memp_malloc(MEMP_UDP_PCB);`.

#### Brief aside on LWIP memory management

It may seem unusual that we would only declare a pointer to a PCB without allocating any memory, at compile-time, for the PCB itself. It may indeed make you very uncomfortable that we are doing what *looks* like a malloc in an embedded program. Might this not lead to memory fragmentation?

LWIP does *not* use standard `malloc`/`free` in order to allocate and  free protocol control blocks or PBUF's. Instead, in the case of PCB's, it statically allocates a *pool* of protocol control block objects. When we runtime-allocate a new udp_pcb, LWIP is actually just grabbing one of the PCB's from this pool. When we free a PCB, LWIP is actually just marking it as "not in use." So, in the case of PCB's, there is not  any dynamic memory allocation occuring at runtime, though the LWIP API vocabulary kind of makes it sound like there is. The same applies to the PBUF structures (though not, as we'll see, to the PBUF payloads). The number of PBUF's in the pool is configurable by way of an LWIP configuration, `PBUF_POOL_SIZE`. And the number of UDP PCB's in the pool is another LWIP configuration, `MEMP_NUM_UDB_PCB`.

For PBUF payloads, LWIP does in fact dynamically allocate memory from heap. But it does so from its own dedicated memory, of a size that we specify in the configuration, and by way of its own implementation of `malloc` that prohibits allocation outside of this dedicated memory. Even so, memory fragmentation is a risk! If you application includes lots of allocating/freeing of different-sized PBUF's, you may encounter this problem. You can mitigate your risk by instead allocating PBUF's from the *PBUF pool*. This pool contains pre-allocated memory for use as PBUF payloads. Using the pool means that your payload size is always the same (though you get to specify what that size is, by way of an LWIP configuration), which means you'll likely be using more memory than you would by allocating. But! You also aren't doing any dynamic memory allocation, which mitigates the risk of fragmentation. Note that all PBUF's allocated for received UDP packets come from this pool.